In [1]:
import gym
import tensorflow as tf

import os

import gym
import numpy as np

from stable_baselines import PPO2
from stable_baselines.bench import Monitor

from stable_baselines.common.policies import ActorCriticPolicy, register_policy, mlp_extractor, MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.common.tf_layers import linear

In [2]:
# Custom MLP policy of three layers of size 128 each for the actor and 2 layers of 32 for the critic,
# with a MLP feature extractor
class CustomPolicyNew(ActorCriticPolicy):
    def __init__(self, sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=False, **kwargs):
        super(CustomPolicyNew, self).__init__(sess, ob_space, ac_space, n_env, n_steps, n_batch, 
                                           reuse=reuse, scale=False)

        with tf.variable_scope("model", reuse=reuse):
            activ = tf.nn.relu

            pi_h, vf_h = mlp_extractor(tf.layers.flatten(self.processed_obs), 
                                               net_arch=[dict(vf=[64, 64], pi=[64, 64])],
                                               act_fun=tf.tanh, **kwargs)
            #extracted_features = tf.layers.flatten(extracted_features)

            #pi_h = extracted_features
            for i, layer_size in enumerate([128, 128, 128]):
                pi_h = activ(tf.layers.dense(pi_h, layer_size, name='pi_fc' + str(i)))
            pi_latent = pi_h

            #vf_h = extracted_features
            for i, layer_size in enumerate([32, 32]):
                vf_h = activ(tf.layers.dense(vf_h, layer_size, name='vf_fc' + str(i)))
            value_fn = tf.layers.dense(vf_h, 1, name='vf')
            vf_latent = vf_h

            self._proba_distribution, self._policy, self.q_value = \
                self.pdtype.proba_distribution_from_latent(pi_latent, vf_latent, init_scale=0.01)

        self._value_fn = value_fn
        self._setup_init()

    def step(self, obs, state=None, mask=None, deterministic=False):
        if deterministic:
            action, value, neglogp = self.sess.run([self.deterministic_action, self.value_flat, self.neglogp],
                                                   {self.obs_ph: obs})
        else:
            action, value, neglogp = self.sess.run([self.action, self.value_flat, self.neglogp],
                                                   {self.obs_ph: obs})
        return action, value, self.initial_state, neglogp

    def proba_step(self, obs, state=None, mask=None):
        return self.sess.run(self.policy_proba, {self.obs_ph: obs})

    def value(self, obs, state=None, mask=None):
        return self.sess.run(self.value_flat, {self.obs_ph: obs})


In [ ]:
class CustomPolicy(ActorCriticPolicy):

    def __init__(self, sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=False, layers=None, 
                 net_arch=None, act_fun=tf.tanh, feature_extraction="mlp", **kwargs):
        super(CustomPolicy, self).__init__(sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=reuse,
                                                scale=(feature_extraction == "cnn"))

        self._kwargs_check(feature_extraction, kwargs)

        if layers is not None:
            warnings.warn("Usage of the `layers` parameter is deprecated! Use net_arch instead "
                          "(it has a different semantics though).", DeprecationWarning)
            if net_arch is not None:
                warnings.warn("The new `net_arch` parameter overrides the deprecated `layers` parameter!",
                              DeprecationWarning)

        if net_arch is None:
            if layers is None:
                layers = [64, 64]
            net_arch = [dict(vf=layers, pi=layers)]

        with tf.variable_scope("model", reuse=reuse):
            pi_latent, vf_latent = mlp_extractor(tf.layers.flatten(self.processed_obs), net_arch, act_fun)

            self._value_fn = linear(vf_latent, 'vf', 1)

            self._proba_distribution, self._policy, self.q_value = \
                self.pdtype.proba_distribution_from_latent(pi_latent, vf_latent, init_scale=0.01)

        self._setup_init()

    def step(self, obs, state=None, mask=None, deterministic=False):
        if deterministic:
            action, value, neglogp = self.sess.run([self.deterministic_action, self.value_flat, self.neglogp],
                                                   {self.obs_ph: obs})
        else:
            action, value, neglogp = self.sess.run([self.action, self.value_flat, self.neglogp],
                                                   {self.obs_ph: obs})
        return action, value, self.initial_state, neglogp

    def proba_step(self, obs, state=None, mask=None):
        return self.sess.run(self.policy_proba, {self.obs_ph: obs})

    def value(self, obs, state=None, mask=None):
        return self.sess.run(self.value_flat, {self.obs_ph: obs})

In [3]:
# Create log dir
log_dir = "tmp/"
os.makedirs(log_dir, exist_ok=True)

In [4]:
env = gym.make('CartPole-v1')
env = Monitor(env, log_dir)

model = PPO2(CustomPolicyNew, env, verbose=1)

Wrapping the env in a DummyVecEnv.




Instructions for updating:
Use keras.layers.flatten instead.

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [5]:
# Train the agent
time_steps = 50000
model.learn(total_timesteps=time_steps)

--------------------------------------
| approxkl           | 9.619993e-06  |
| clipfrac           | 0.0           |
| ep_len_mean        | 21.8          |
| ep_reward_mean     | 21.8          |
| explained_variance | 0.0345        |
| fps                | 202           |
| n_updates          | 1             |
| policy_entropy     | 0.6931372     |
| policy_loss        | -0.0006388412 |
| serial_timesteps   | 128           |
| time_elapsed       | 4.43e-05      |
| total_timesteps    | 128           |
| value_loss         | 46.837788     |
--------------------------------------
--------------------------------------
| approxkl           | 1.3510906e-05 |
| clipfrac           | 0.0           |
| ep_len_mean        | 23            |
| ep_reward_mean     | 23            |
| explained_variance | 0.00579       |
| fps                | 818           |
| n_updates          | 2             |
| policy_entropy     | 0.6930676     |
| policy_loss        | -0.0009091749 |
| serial_timesteps   | 25

---------------------------------------
| approxkl           | 0.00017307905  |
| clipfrac           | 0.0            |
| ep_len_mean        | 24.8           |
| ep_reward_mean     | 24.8           |
| explained_variance | 0.0105         |
| fps                | 869            |
| n_updates          | 16             |
| policy_entropy     | 0.6405998      |
| policy_loss        | -0.00078717695 |
| serial_timesteps   | 2048           |
| time_elapsed       | 2.81           |
| total_timesteps    | 2048           |
| value_loss         | 63.797295      |
---------------------------------------
-------------------------------------
| approxkl           | 0.0006060024 |
| clipfrac           | 0.0          |
| ep_len_mean        | 26           |
| ep_reward_mean     | 26           |
| explained_variance | -0.163       |
| fps                | 865          |
| n_updates          | 17           |
| policy_entropy     | 0.6620505    |
| policy_loss        | 0.0034306026 |
| serial_timesteps  

--------------------------------------
| approxkl           | 0.00094983005 |
| clipfrac           | 0.00390625    |
| ep_len_mean        | 38.7          |
| ep_reward_mean     | 38.7          |
| explained_variance | -0.0291       |
| fps                | 848           |
| n_updates          | 31            |
| policy_entropy     | 0.62005264    |
| policy_loss        | 0.007616415   |
| serial_timesteps   | 3968          |
| time_elapsed       | 5.08          |
| total_timesteps    | 3968          |
| value_loss         | 64.04257      |
--------------------------------------
--------------------------------------
| approxkl           | 0.00012561076 |
| clipfrac           | 0.0           |
| ep_len_mean        | 39.8          |
| ep_reward_mean     | 39.8          |
| explained_variance | -0.0316       |
| fps                | 817           |
| n_updates          | 32            |
| policy_entropy     | 0.61019146    |
| policy_loss        | 0.00086507597 |
| serial_timesteps   | 40

-------------------------------------
| approxkl           | 0.0024790657 |
| clipfrac           | 0.029296875  |
| ep_len_mean        | 52           |
| ep_reward_mean     | 52           |
| explained_variance | 0.29         |
| fps                | 811          |
| n_updates          | 46           |
| policy_entropy     | 0.6263214    |
| policy_loss        | -0.002653458 |
| serial_timesteps   | 5888         |
| time_elapsed       | 7.38         |
| total_timesteps    | 5888         |
| value_loss         | 71.89594     |
-------------------------------------
--------------------------------------
| approxkl           | 0.0016910178  |
| clipfrac           | 0.009765625   |
| ep_len_mean        | 54.5          |
| ep_reward_mean     | 54.5          |
| explained_variance | 0.242         |
| fps                | 850           |
| n_updates          | 47            |
| policy_entropy     | 0.55426717    |
| policy_loss        | -0.0029462145 |
| serial_timesteps   | 6016          |
|

-------------------------------------
| approxkl           | 0.0005953453 |
| clipfrac           | 0.0          |
| ep_len_mean        | 69.7         |
| ep_reward_mean     | 69.7         |
| explained_variance | 0.088        |
| fps                | 866          |
| n_updates          | 61           |
| policy_entropy     | 0.5571782    |
| policy_loss        | -0.004538995 |
| serial_timesteps   | 7808         |
| time_elapsed       | 9.7          |
| total_timesteps    | 7808         |
| value_loss         | 47.046402    |
-------------------------------------
--------------------------------------
| approxkl           | 0.0026700404  |
| clipfrac           | 0.015625      |
| ep_len_mean        | 70.9          |
| ep_reward_mean     | 70.9          |
| explained_variance | 0.115         |
| fps                | 829           |
| n_updates          | 62            |
| policy_entropy     | 0.5797389     |
| policy_loss        | -0.0030489974 |
| serial_timesteps   | 7936          |
|

...

---------------------------------------
| approxkl           | 0.00096515566  |
| clipfrac           | 0.001953125    |
| ep_len_mean        | 255            |
| ep_reward_mean     | 255            |
| explained_variance | -1.09          |
| fps                | 808            |
| n_updates          | 329            |
| policy_entropy     | 0.3376761      |
| policy_loss        | -0.00033945846 |
| serial_timesteps   | 42112          |
| time_elapsed       | 52.8           |
| total_timesteps    | 42112          |
| value_loss         | 0.020389628    |
---------------------------------------
-------------------------------------
| approxkl           | 0.005079034  |
| clipfrac           | 0.06640625   |
| ep_len_mean        | 255          |
| ep_reward_mean     | 255          |
| explained_variance | 0.49         |
| fps                | 847          |
| n_updates          | 330          |
| policy_entropy     | 0.3565007    |
| policy_loss        | 0.0015751992 |
| serial_timesteps  

--------------------------------------
| approxkl           | 3.963812e-05  |
| clipfrac           | 0.0           |
| ep_len_mean        | 272           |
| ep_reward_mean     | 272           |
| explained_variance | -1.48         |
| fps                | 820           |
| n_updates          | 344           |
| policy_entropy     | 0.36055157    |
| policy_loss        | 0.00015786214 |
| serial_timesteps   | 44032         |
| time_elapsed       | 55.2          |
| total_timesteps    | 44032         |
| value_loss         | 0.021143485   |
--------------------------------------
-------------------------------------
| approxkl           | 0.011548545  |
| clipfrac           | 0.11328125   |
| ep_len_mean        | 272          |
| ep_reward_mean     | 272          |
| explained_variance | -11.3        |
| fps                | 782          |
| n_updates          | 345          |
| policy_entropy     | 0.32837707   |
| policy_loss        | -0.011552495 |
| serial_timesteps   | 44160       

--------------------------------------
| approxkl           | 0.010553232   |
| clipfrac           | 0.109375      |
| ep_len_mean        | 286           |
| ep_reward_mean     | 286           |
| explained_variance | 0.459         |
| fps                | 817           |
| n_updates          | 359           |
| policy_entropy     | 0.43330765    |
| policy_loss        | -0.0021989846 |
| serial_timesteps   | 45952         |
| time_elapsed       | 57.7          |
| total_timesteps    | 45952         |
| value_loss         | 208.14789     |
--------------------------------------
------------------------------------
| approxkl           | 0.015562485 |
| clipfrac           | 0.20898438  |
| ep_len_mean        | 286         |
| ep_reward_mean     | 286         |
| explained_variance | 0.572       |
| fps                | 828         |
| n_updates          | 360         |
| policy_entropy     | 0.39136028  |
| policy_loss        | -0.01523221 |
| serial_timesteps   | 46080       |
| time_e

--------------------------------------
| approxkl           | 0.0017997982  |
| clipfrac           | 0.017578125   |
| ep_len_mean        | 302           |
| ep_reward_mean     | 302           |
| explained_variance | -0.00325      |
| fps                | 812           |
| n_updates          | 374           |
| policy_entropy     | 0.36667144    |
| policy_loss        | -0.0032008376 |
| serial_timesteps   | 47872         |
| time_elapsed       | 60.1          |
| total_timesteps    | 47872         |
| value_loss         | 328.798       |
--------------------------------------
---------------------------------------
| approxkl           | 0.0009239847   |
| clipfrac           | 0.01171875     |
| ep_len_mean        | 302            |
| ep_reward_mean     | 302            |
| explained_variance | -11.3          |
| fps                | 817            |
| n_updates          | 375            |
| policy_entropy     | 0.38312992     |
| policy_loss        | -0.00053465995 |
| serial_timest

-------------------------------------
| approxkl           | 0.002168411  |
| clipfrac           | 0.021484375  |
| ep_len_mean        | 312          |
| ep_reward_mean     | 312          |
| explained_variance | -0.549       |
| fps                | 832          |
| n_updates          | 389          |
| policy_entropy     | 0.42412362   |
| policy_loss        | 0.0029362175 |
| serial_timesteps   | 49792        |
| time_elapsed       | 62.4         |
| total_timesteps    | 49792        |
| value_loss         | 0.042799614  |
-------------------------------------
-------------------------------------
| approxkl           | 0.015358709  |
| clipfrac           | 0.1875       |
| ep_len_mean        | 315          |
| ep_reward_mean     | 315          |
| explained_variance | 0.00172      |
| fps                | 789          |
| n_updates          | 390          |
| policy_entropy     | 0.39766082   |
| policy_loss        | 0.0023935658 |
| serial_timesteps   | 49920        |
| time_elaps

In [6]:
# Enjoy trained agent
obs = env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=False)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
        obs = env.reset()
env.close()